# Data exploration in the PSA with EPN-TAP
- EPN-TAP is a Europlanet extension of TAP for planetary science
- Remember: the PSA is intrisincally multi-mission, multi-instrument archive
- The PSA supports access to products via EPN-TAP
  - EPN-TAP allows queries to the database not possible with other interfaces
- The PSA TAP database offers a _catalogue_ of data products, not the data themselves
- Data in the PSA are archived in [PDS](https://pds.nasa.gov/) format
    - each archive item consists of product + label
        - PDS3 = previous standard (labels in PVL)
        - PDS4 = current standard (labels in XML)
- PSA data are highly diverse
    - from microscope to telescopes
    - on orbiters, flyby missions, landers etc.
- The PSA supports versioning of products/datasets and private data
    - PSA EPN-TAP shows only the latest version
    - Meta-data for private data are shown, but no detailed info
        - so calculations of product _size_ apply only to public data, for example
- EPN-TAP indexes __only__ the data/observational products
    - not calibration files, documents, ancillary products etc.

In [2]:
import zipfile, struct, warnings, requests, struct, os
from pathlib import PurePath
from io import BytesIO

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import astropy, skimage
from astropy.io.votable import parse_single_table
from astroquery.utils.tap.core import Tap
import pvl

from pds4_tools import read

import IPython.display as display
from PIL import Image

%matplotlib inline
warnings.simplefilter('ignore')

Point the astroquery TAP library to the PSA EPN-TAP server

In [3]:
tap_url = 'https://archives.esac.esa.int/psa/epn-tap/tap/'
psa = Tap(url=tap_url)

def query(q):
    return(psa.launch_job(q).get_data())

## PSA data exploration

Run some simple queries to get some statistics about the data in the PSA

### How many missions does the PSA serve?

In [4]:
query('SELECT DISTINCT instrument_host_name FROM epn_core')

instrument_host_name
object
Huygens
Venus Express
Mars Express
BepiColombo
Giotto
Ground Based
ExoMars 2016
Hubble
SMART-1


### ...and how many instruments?

In [8]:
len(query('SELECT DISTINCT instrument_name FROM epn_core'))

78

### How many products are there in the archive?

In [6]:
result = query('SELECT COUNT(1) from epn_core')
result['count'][0]

9557830

### How many products are there per mission?

In [ ]:
 query('SELECT instrument_host_name, count(*) as num_prod FROM epn_core GROUP BY instrument_host_name ORDER BY num_prod DESC')

### And how *much* data (in TB)?

In [ ]:
result = query('SELECT SUM(access_estsize) from epn_core') # in kbyte
print('Size of latest versions of public data: {:3.2f} TB'.format(result['sum'][0]/1024.**3))

### What are the biggest products?

In [ ]:
query('SELECT TOP 10 granule_uid, access_estsize from epn_core WHERE access_estsize IS NOT NULL ORDER BY access_estsize DESC')

### Which mission has the most data?

In [ ]:
query('SELECT instrument_host_name, sum(access_estsize)/1024./1024. as prod_size from epn_core WHERE access_estsize IS NOT NULL GROUP BY instrument_host_name ORDER BY prod_size DESC')

Wow, so the top two missions account for almost all of the data in the archive:

### How many different targets are there in the archive?

In [ ]:
query('SELECT DISTINCT target_name FROM epn_core')

# Diving into the data

## OK, what data do we have about Mars - show me the missions!

In [ ]:
query("SELECT DISTINCT instrument_host_name FROM epn_core WHERE target_name='Mars'")

## Interesting - Rosetta observed Mars! With which instruments?

In [ ]:
query("SELECT DISTINCT instrument_name FROM epn_core WHERE target_name='Mars' AND instrument_host_name='Rosetta'")

## OSIRIS is the camera, right? What data did it produce?

In [ ]:
osiris_mars = query("SELECT * FROM epn_core WHERE target_name='Mars' AND instrument_name='OSIRIS'")
len(osiris_mars)

In [ ]:
osiris_mars.colnames

In [ ]:
osiris_mars[0]

## Let's just tidy that up a bit

- Putting the table into pandas since I'm more comfortable with it
- There are lots of columns with "Nan"s because not all fields can be (easily) populated due to the diverse nature of the instruments
    - remove those 
- convert Julian times into readable timestamps
- resulting function: query_pandas(query)

In [ ]:
def query_pandas(q):
    df = query(q).to_pandas()
    df.dropna(axis=1, how='all', inplace=True)
    str_df = df.select_dtypes([np.object])
    if not str_df.empty:
        str_df = str_df.stack().str.decode('utf-8').unstack()
        for col in str_df:
            df[col] = str_df[col]        
    df['time_min'] = pd.to_datetime(df['time_min'], origin='julian', unit='D') 
    df['time_max'] = pd.to_datetime(df['time_max'], origin='julian', unit='D') 
    return df

In [ ]:
osiris_mars = query_pandas("SELECT * FROM epn_core WHERE target_name='Mars' AND instrument_name='OSIRIS'")

Now we have all of the fields that have populated values in this query - let's look at the first row:

In [ ]:
osiris_mars.iloc[0]

Ahh, thumbnails are provided!

Great, I don't have to work hard to get a sneak preview! Let's look at an example - we can show images directly in the Notebook:

In [ ]:
osiris_mars.iloc[0].thumbnail_url

In [ ]:
display.Image(osiris_mars.thumbnail_url.iloc[0], width=500, embed=True)

We can also grab a bunch of images and show them using matplotlib (easier to tile etc.). Let's take the first 9:

## Let's get the _real_ data for that image

I have a URL for direct access, and a format - which the MIME code tell me is a zip.

In [ ]:
access_url = osiris_mars.iloc[0].access_url
access_url

Let's download the zip on the fly and see what's in it...

In [ ]:
r = requests.get(url=access_url)
zip = zipfile.ZipFile(BytesIO(r.content))
zipfiles = zip.namelist()
zipfiles

So the product actually consists of the image file with attached label, and a browse product. We need to get the file in the DATA directory to work with the actual data...


In [ ]:
data_file = [file for file in zipfiles[:-1] if PurePath(file).parts[1]=='DATA'][0]
data_file

Ooh, attached PDS3 labels are nasty - I picked a bad example! Still, we can deal with it - find the label length by parsing until we hit "END" in function read_label()

In [ ]:
def read_label(data_file):
    label = []
    with zip.open(data_file) as f:  
        line = f.readline().decode('utf8')
        while line.strip() != 'END':
            if line.strip() == '':
                line = f.readline().decode('utf8')
                continue
            else:
                label.append(line.strip())
                line = f.readline().decode('utf8')
            
    return '\r\n'.join(label)

In [ ]:
label = read_label(data_file)

PDS3 labels are stored in "Parameter Value Language" (PVL) which we can pass to get a dictionary...

In [ ]:
meta = pvl.loads(label)
meta

Oof, that's a lot of meta-data! Now we have all of the PDS3 meta-data ready to be used. We can use the IMAGE pointer to tell us where to get the actual image array

In [ ]:
offset = (meta['^IMAGE']-1) * meta['RECORD_BYTES']
offset

But we need a bit more meta-data to open it correctly...

In [ ]:
meta['IMAGE']

In [ ]:
rows = meta['IMAGE']['LINES']
cols = meta['IMAGE']['LINE_SAMPLES']
samples = rows * cols

OK, how we have enough to unpack the data!

In [ ]:
data = struct.unpack_from('<%dH' % samples, zip.open(data_file).read(), offset)
data = np.array(data).reshape((cols, rows))

and plot them:

In [ ]:
fig, ax = plt.subplots(figsize=(14,14))
im = ax.imshow(data, cmap=matplotlib.cm.gray, interpolation='nearest')

Check the minimum, maximum and mean values to compare with the label:

In [ ]:
meta['IMAGE']['DERIVED_MINIMUM'], meta['IMAGE']['DERIVED_MAXIMUM'], meta['IMAGE']['MEAN']

In [ ]:
data.min(), data.max(), data.mean()

Now we have the real data, not just a preview, and can try things like histograms

In [ ]:
fig_hist, ax_hist = plt.subplots(figsize=(14,10))
ax_hist.hist(data.ravel(), bins=256, histtype='step', color='black')
ax_hist.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))
ax_hist.set_xlabel('Pixel intensity')
ax_hist.set_yticks([]);

Now we can play with things like adaptive histogram equalisation:

In [ ]:
img_adapteq = skimage.exposure.equalize_adapthist(data, clip_limit=0.03)
fig, ax = plt.subplots(figsize=(14,14))
im = ax.imshow(img_adapteq, cmap=matplotlib.cm.gray, interpolation='nearest')

# PDS4

Can we check if a data product is PDS3 or PDS4 in advance? Kind of... The granule_gid for PDS4 products will always start "urn:esa:psa" - this is the prefix of the LID (logical identifier) in PDS4. For PDS3 this attribute will start with the dataset name. Let's see which missions produce which data type:

In [ ]:
query("SELECT DISTINCT instrument_host_name from epn_core WHERE granule_gid LIKE 'urn:esa:psa:%'")

In [ ]:
query("SELECT DISTINCT instrument_host_name from epn_core WHERE granule_gid NOT LIKE 'urn:esa:psa:%'")

PDS4 data are easier to work with since we have nice tools that work on all valid products. Let's see if BepiColombo has any public data - we can check if data are public by seeing if they have a download URL:

In [ ]:
query("SELECT DISTINCT instrument_name from epn_core WHERE instrument_host_name='BepiColombo' and access_url IS NOT NULL")

OK, the monitoring camera has public data - let's take a look!

In [ ]:
mcam = query_pandas("SELECT * from epn_core WHERE instrument_name='MCAM' and access_url IS NOT NULL")
len(mcam)

In [ ]:
mcam.iloc[-1]

BepiColombo is using the PSA as an "operational" archive, and the MCAM data are made public ~a week after they hit the ground, so the last public data are only ~10 days old...

In [ ]:
r = requests.get(url=mcam.iloc[-1].access_url)
zip = zipfile.ZipFile(BytesIO(r.content))
zipfiles = zip.namelist()
zipfiles

Here we get the browse image, and a FITS file with the data. We could access this with a FITS library, but since it is fully described by a PDS4 label, we can also use a PDS4 viewer, which would work on _any_ PDS4 product. Reading only the fly is a bit messy, so here I will just extract the files locally:

In [ ]:
zip.extract(zipfiles[0])
zip.extract(zipfiles[1])

Point the PDS4 reader at the label and you will get all of the data structures returned:

In [ ]:
mcam_data = read(zipfiles[0])

In [ ]:
mcam_data.info()

Now we can simply plot the data from the Array_2D_Image structure:

In [ ]:
fig, ax = plt.subplots(figsize=(14,14))
im = ax.imshow(mcam_data['MCAM_image'].data, cmap=matplotlib.cm.gray, interpolation='nearest', origin='top')